In [1]:
# search in the hyperparameter space with W&B sweep
import logging
from ast import literal_eval as load
import pandas as pd
import numpy as np
from ast import literal_eval
import sklearn

from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)


In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df = pd.read_csv('triples.csv')
df = df.rename(columns={'idx': 'indx'})
df.insert(loc=0, column='idx', value=np.arange(len(df)))
sent_num=len(df)

In [3]:
def convert(arr):
    trip_ls = []
    ls = []
    for i in range(len(arr)):
        pre_ls = literal_eval(arr[i, 3])
        np_ls = literal_eval(arr[i, 4])
        p_idx = [(1, *p[1]) for p in pre_ls]
        n_idx = [(0, *n[1]) for n in np_ls]
        for p in range(len(p_idx)):
            for j in range(len(n_idx)-1):
                for k in range(j+1, len(n_idx)):
                    trip = [np_ls[j][0],pre_ls[p][0],np_ls[k][0]]
                    trip_ls.append(trip)
                    word_ls = arr[i, 2].split(' ')
                    indx = sorted([p_idx[p], n_idx[j], n_idx[k]], key=lambda x:x[1])
                    for w in range(len(indx)):
                        if indx[w][0] == 1:
                            word_ls.insert(indx[w][1]+2*w, '<<')
                            word_ls.insert(indx[w][2]+2*w+1, '>>')
                        else:
                            word_ls.insert(indx[w][1]+2*w, '[[')
                            word_ls.insert(indx[w][2]+2*w+1, ']]')
                    flg = 0
                    for tp in literal_eval(arr[i, 5]):
                        if tp == [np_ls[j][0], pre_ls[p][0], np_ls[k][0]]:
                            flg = 1
                            break
                    ls.append([int(arr[i, 0]), ' '.join(word_ls), flg])
    dataframe = pd.DataFrame(ls)
    dataframe.columns = ['idx', 'text', 'labels']
    return dataframe,trip_ls

In [4]:
df=df.values
df,trip_ls=convert(df)

In [5]:
model_args = ClassificationArgs()

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.manual_seed = 1
model_args.fp16 = False
model_args.use_multiprocessing = True
model_args.do_lower_case = True  # when using uncased model

In [6]:
def triple_F1(ref, pred):
    return 0

In [7]:
# Create a TransformerModel
model = ClassificationModel(
    "bert",
    "../rel/outputsA/best_model",
    args=model_args,
)
result, model_outputs, wrong_predictions = model.eval_model(df, F1_score=triple_F1)

    

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/103636 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/12955 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 0, 'tn': 96801, 'fp': 6835, 'fn': 0, 'F1_score': 0, 'eval_loss': 0.32792378466614797}


In [20]:
preds = list(model_outputs.argmax(axis=1))
df['preds']=preds
df['cand']=trip_ls
df.loc[df['preds']==0,'cand']=None
data=[]
for i in range(sent_num):
    temp = list(df[df['idx']==i]['cand'])
    temp = [t for t in temp if t]
    data.append(str(temp))



In [21]:
data

['[]',
 "[['sentences', 'explicitly models', 'nested , hierarchical relationships'], ['nested , hierarchical relationships', 'among', 'words and phrases']]",
 "[['two variants', 'of', 'algorithm'], ['two variants', 'one for', 'parsing'], ['algorithm', 'one for', 'parsing']]",
 "[['greedy prediction', 'with', 'our']]",
 "[['simple importance sampling algorithm', 'which uses', 'samples'], ['samples', 'from', 'discriminative parser'], ['samples', 'to solve', 'inference problems'], ['discriminative parser', 'to solve', 'inference problems'], ['inference problems', 'in', 'generative model']]",
 '[]',
 "[['training', 'used', 'stochastic gradient descent'], ['stochastic gradient descent', 'with', 'learning rate'], ['learning rate', 'of', '0.1']]",
 '[]',
 '[]',
 '[]',
 "[['both directions', 'of', 'large language - model - inspired self - attention cloze model']]",
 "[['cloze - style training objective', 'where', 'model'], ['model', 'must predict', 'center word'], ['center word', 'given', 'lef

In [22]:
data=pd.DataFrame(data,columns=['triple_A'])
data

,triple_A
0,[]
1,"[['sentences', 'explicitly models', 'nested , ..."
2,"[['two variants', 'of', 'algorithm'], ['two va..."
3,"[['greedy prediction', 'with', 'our']]"
4,"[['simple importance sampling algorithm', 'whi..."
...,...
3144,"[['short and long answers', 'in', 'single mode..."
3145,[]
3146,"[['our model', 'from', 'BERT model'], ['our mo..."
3147,"[['model', 'by', 'minimizing'], ['loss L', 'wi..."


In [19]:
s=[[[4],6]]
str(s)

'[[[4], 6]]'

In [23]:
data

,triple_A
0,[]
1,"[['sentences', 'explicitly models', 'nested , ..."
2,"[['two variants', 'of', 'algorithm'], ['two va..."
3,"[['greedy prediction', 'with', 'our']]"
4,"[['simple importance sampling algorithm', 'whi..."
...,...
3144,"[['short and long answers', 'in', 'single mode..."
3145,[]
3146,"[['our model', 'from', 'BERT model'], ['our mo..."
3147,"[['model', 'by', 'minimizing'], ['loss L', 'wi..."


In [24]:
data.to_csv('A.csv',index=False)